<a href="https://colab.research.google.com/github/JabbaSciMan/FAILEDchatGPTledstrips5gamespaceinvaders/blob/master/Canvas_2_Zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""

if no score:
    if attempted --> full credit
    elif Hyrum Z --> EX
    else: --> Zero
elif ungraded resubmission --> FC

# I can change the assigments to "bucket ungraded"

This may be more modularized and better.  Made by ChatGPT

https://chatgpt.com/c/675685cb-7834-800d-ba7e-df6ed44ab94f

z GameDev79301.txt
z Physics7928.txt
z WebDev79301.txt
"""

####################################################################################################
# only zeroedAssignments_79301 is getting added to even if I shorten the list of canvas course ID's.
########################################################################################################
!pip install colorama
!pip install canvasapi # Install the canvasapi module
from time import *

from datetime import datetime, timedelta
import colorama
from canvasapi import Canvas
colorama.init(autoreset=True)

CANVAS_API_URL = 'https://ccsdut.instructure.com'
ACCESS_TOKEN = '3776~iunsQDOjYR5U3hdlOUM04cC7jtbXj9OUzJt5a9L8mUKzIywR7a9JhqJWrWTui4U8'

#choice = int(input("\n\n\nEnter 0 for all assignemnts, Enter 1 for Ungraded Only Assignments (faster).:  "))
                                 #[79301,79494,79298]
currentCanvasCourses = [79301,79494,79298]

# create text files for each course for already Zeroed assignments
# if the file already exists, it won't create a new one or write over the top
for COURSE_ID in currentCanvasCourses:
    file_name = f"zeroedAssignments_{COURSE_ID}.txt"
    f = open(file_name, "a")
    f.close()

for COURSE_ID in currentCanvasCourses:
    st = time()
    to_be_added_list = []
    #create a list of already zero'd assignments
    file_name = f"zeroedAssignments_{COURSE_ID}.txt"
    with open(file_name, "r") as file:
        zeroList = [line.strip() for line in file]
        # now I have a zeroList


    curdate = (datetime.today() + timedelta(1)).strftime("%Y-%m-%d") + "T05:59:59Z"

    canvas = Canvas(CANVAS_API_URL, ACCESS_TOKEN)
    course = canvas.get_course(COURSE_ID)

    # loop twice.  The first time is to grade the ungraded.
    # the 2nd loop through is to give zero's for no scores and loops through published assignments not in the zeroed list.

    for i in range(2):
        if i==0:
            assignments = course.get_assignments(bucket='ungraded')
            print("List of Ungraded Assignments:    \n\n")
        else:
            assignments = course.get_assignments()
            print("List of Published Assignments:    \n\n")


        students = course.get_users(enrollment_type=['student'])



        def printPublishedAssignmentNames(assignments):
            assignmentNamesList =[]
            for assignment in assignments:
                if assignment.published:
                    assignmentNamesList.append(assignment.name)
            assignmentNamesList.sort()
            for each in assignmentNamesList:
                print(each)
        printPublishedAssignmentNames(assignments)

    ####


       ####
        for assignment in assignments:

            if assignment.published and assignment.assignment_group_id != 144114 and (assignment.name not in zeroList or i==0): # i was choice  not i
                #print(assignment.assignment_group_id)
                #144114 - Imported Unity Ass. Group in Game Dev class.
                # if assign published and (either unpublished or not in Zero's list)

                if len(assignment.name)>49:
                    assignment.edit(assignment={'name': assignment.name[:50]})
                assignment.edit(assignment = {"post_to_sis": True})
                print(f"{colorama.Fore.BLUE}Assignment: {assignment.name}")
                print("")
                #zig
                if i==1:
                    to_be_added_list.append(assignment.name)
            #set due date if assign does not have one.
                if not assignment.due_at:
                    assignment.edit(assignment = {"due_at": curdate})
                    #print("Changed due date to midnight tonight.\n")
                    #print("")
                print(assignment)

                for student in students:
                    current_submission = assignment.get_submission(student.id)
                    ungraded_Resubmission = not current_submission.grade_matches_current_submission
                    if i==0 and current_submission.score != None and not ungraded_Resubmission:
                        #print("Submission is submitted but ungraded.")
                        continue

                    #give zero's for no submission
                    #if they have a score and resubmitted is false then Continue#####*****
                    if current_submission.excused:
                        continue
                    if current_submission.score == None:
                        if current_submission.attempt:
                            current_submission.edit(submission = {"posted_grade": "complete"})
                            print(f"{colorama.Fore.YELLOW}Student: {student.name}")
                            print(f"{colorama.Fore.YELLOW}Assignment graded & given 100%.")
                            print(f"{colorama.Fore.GREEN}Assignment: {assignment.name}")
                            print(f'{colorama.Fore.RED}New score: {current_submission.score}')
                            print("")

                        elif student.name in ["Hyrum Zaugg", "Jaxson Howe"] and COURSE_ID == 7777: # specific course 2 come
                            current_submission.edit(submission = {"excuse": True})
                            print("excused one")
                            print("")

                        ########################
                        ################  https://chatgpt.com/c/6761ab99-64f8-800d-9689-2a280c27c3c8
                        # at the bottom of the link above, chatgpt shows how to compare different time formats just in case they are not in the same format.
                        elif assignment.due_at > curdate:
                            continue

                        else:
                            current_submission.edit(submission = {"posted_grade": 0})
                            print(f"{colorama.Fore.BLUE}Student: {student.name}")
                            print(f"{colorama.Fore.GREEN}Assignment: {assignment.name}")
                            print(f'{colorama.Fore.RED}New score: {current_submission.score}')
                            print("")
                    elif ungraded_Resubmission:
                        current_submission.edit(submission = {"posted_grade": "complete"})
                        print(f"{colorama.Fore.YELLOW}Student: {student.name}")
                        print(f"{colorama.Fore.YELLOW}Assignment graded & given 100%.")
                        print(f"{colorama.Fore.GREEN}Assignment: {assignment.name}")
                        print(f'{colorama.Fore.RED}New score: {current_submission.score}')
                        print("Ungraded_Resubmission worked and it really did work because that is so cool since this might be the first time.")
                    #else:
                    #    print(assignment.name, student.name,"skipped - has score already.")



    # this appends the new list of names to the file.
    with open(file_name, "a") as file:
        for assignmentName in to_be_added_list:
            file.write(assignmentName + "\n")

    et = time()-st
    min = int(et // 60)
    sec = et % 60

    print(round(et,3), "seconds passed")
    print(min,"minutes and",round(sec,1),"seconds passed")
    xxx = str(min) + " minutes and " + str(round(sec,1)) + " seconds passed"

    file_name = f"time_records_{file_name}.txt"
    with open(file_name, "a") as file:
        file.write(xxx + "\n")





        # I can append the date and times the file has finished and the time it took.

List of Ungraded Assignments:    


Unit 1 Quiz: Unity fundamentals
List of Published Assignments:    


1st Kami
20 signatures box game
Assignment: Add lighting to your game
Assignment: Add some magic to your scene
Assignment: Bring the scene to life
Assignment: Building the game
Assignment: Create a balanced primitive structure
Assignment: Create your still life composition
Assignment: Finalize your project
Assignment: Make a worldspace UI
Assignment: Your own soundscape
Attendance Score
Box Game Hex file.
Box Game Link
Box Game Video
Chrome Bookmarks
EC Screenshot Canvas App on Phone
Edpuzzle | Diegetic vs Nondiegetic Sound, Part 1
Essential Keyboard Shortcuts
Game Dev Syllabus Page
On Task Score
Pass off the LED strip game
Prove you can use Unity Version Control
Publish a Unity Game
Snowman spiral Fidget EC
Submit Maze Game
Submit Screenshot of over 200%
Submit gif file created from PiskelApp.com
Syllabus Review Questions
Tailor the LED game
Unit 1 Quiz: Unity fundamentals
Unit 2 Q